Import packages

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm
from joblib import Parallel, delayed
from tqdm import tqdm
import time

Load data

In [4]:
rf = 0.002

market = pd.read_csv("MarketExcessReturns.csv", header=None).values.flatten()
returns = pd.read_csv("MonthlyExcessReturns.csv", header=None)
mu_pop = pd.read_csv("ExpectedExcessReturns.csv", header=None).values.flatten()
Sigma_pop = pd.read_csv("CovarianceMatrix.csv", header=None).values

T, N = returns.shape

i. Compute Sharpe ratio of the market

In [6]:
mean_return = np.mean(market)
std_return = np.std(market, ddof=0)
sharpe_ratio_market = (mean_return - rf) / std_return
print(f"Sharpe Ratio of Market Portfolio: {sharpe_ratio_market:.4f}")

Sharpe Ratio of Market Portfolio: 0.1290


ii. Sharpe ratio of the 1/N portfolio

In [8]:
w_1N = np.ones(N) / N
mu_1N = np.dot(w_1N, mu_pop)
sigma_1N = np.sqrt(w_1N @ Sigma_pop @ w_1N)
sharpe_1N = (mu_1N - rf) / sigma_1N

print(f"Sharpe Ratio of 1/N Portfolio: {sharpe_1N:.4f}")

Sharpe Ratio of 1/N Portfolio: 0.1180


iii. Sharpe Ratio of the Mean-Variance Optimal Portfolio

In [10]:
mu_sample = returns.mean().values
Sigma_sample = returns.cov().values
inv_Sigma_sample = np.linalg.inv(Sigma_sample)

w_mv_sample = inv_Sigma_sample @ mu_sample / (np.ones(N) @ inv_Sigma_sample @ mu_sample)
mu_mv_sample = w_mv_sample @ mu_pop
sigma_mv_sample = np.sqrt(w_mv_sample @ Sigma_pop @ w_mv_sample)
sharpe_mv_sample = (mu_mv_sample - rf) / sigma_mv_sample

print(f"Sharpe Ratio of Mean-Variance Optimal Portfolio: {sharpe_mv_sample:.4f}")

Sharpe Ratio of Mean-Variance Optimal Portfolio: 0.0466


iv. Sharpe Ratio of the Global Minimum Variance Portfolio

In [12]:
w_min_sample = inv_Sigma_sample @ np.ones(N) / (np.ones(N) @ inv_Sigma_sample @ np.ones(N))
mu_min_sample = w_min_sample @ mu_pop
sigma_min_sample = np.sqrt(w_min_sample @ Sigma_pop @ w_min_sample)
sharpe_min_sample = (mu_min_sample - rf) / sigma_min_sample

print(f"Sharpe Ratio of Global Minimum Variance Portfolio: {sharpe_min_sample:.4f}")

Sharpe Ratio of Global Minimum Variance Portfolio: 0.0078


v. Sharpe Ratio Using James-Stein-Jorion Shrinkage Estimator

In [14]:
# minimum variance portfolio return
mu_0 = (np.ones(N) @ inv_Sigma_sample @ mu_sample) / (np.ones(N) @ inv_Sigma_sample @ np.ones(N))

# shrinkage intensity
diff = mu_sample - mu_0
v_num = N + 2
v_den = N + 2 + T * diff.T @ inv_Sigma_sample @ diff
v = v_num / v_den

# James-Stein-Jorion shrunk mean
mu_jsj = (1 - v) * mu_sample + v * mu_0 * np.ones(N)

# Optimal weights using shrunk means
w_jsj = inv_Sigma_sample @ mu_jsj / (np.ones(N) @ inv_Sigma_sample @ mu_jsj)

mu_jsj_opt = w_jsj @ mu_pop
sigma_jsj_opt = np.sqrt(w_jsj @ Sigma_pop @ w_jsj)
sharpe_jsj = (mu_jsj_opt - rf) / sigma_jsj_opt

print(f"Sharpe Ratio with James-Stein-Jorion Estimator: {sharpe_jsj:.4f}")

Sharpe Ratio with James-Stein-Jorion Estimator: 0.0443


vi. Sharpe Ratio Using CAPM Expected Returns

In [16]:
betas = np.array([
    sm.OLS(returns.iloc[:, i], sm.add_constant(market)).fit().params[1]
    for i in range(N)
])

mu_market = np.mean(market)
mu_CAPM = rf + betas * (mu_market - rf)

w_CAPM = inv_Sigma_sample @ mu_CAPM / (np.ones(N) @ inv_Sigma_sample @ mu_CAPM)

mu_CAPM_opt = w_CAPM @ mu_pop
sigma_CAPM_opt = np.sqrt(w_CAPM @ Sigma_pop @ w_CAPM)
sharpe_CAPM = (mu_CAPM_opt - rf) / sigma_CAPM_opt

print(f"Sharpe Ratio using CAPM: {sharpe_CAPM:.4f}")

Sharpe Ratio using CAPM: 0.0827


vii. Sharpe Ratio with No Short-Selling

In [18]:
def minimize_variance(target_mu, mu_estimate, Sigma_estimate):
    def portfolio_variance(w):
        return w @ Sigma_estimate @ w
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w: w @ mu_estimate - target_mu}
    ]
    bounds = [(0, None)] * N
    initial_w = np.ones(N) / N
    result = minimize(portfolio_variance, initial_w, bounds=bounds, constraints=constraints)
    return result.x if result.success else None

target_returns = np.linspace(mu_sample.min(), mu_sample.max(), 100)
best_sharpe = -np.inf
best_w = None

for target_mu in target_returns:
    w_candidate = minimize_variance(target_mu, mu_sample, Sigma_sample)
    if w_candidate is not None:
        mu_true = w_candidate @ mu_pop
        sigma_true = np.sqrt(w_candidate @ Sigma_pop @ w_candidate)
        sharpe_candidate = (mu_true - rf) / sigma_true
        if sharpe_candidate > best_sharpe:
            best_sharpe = sharpe_candidate
            best_w = w_candidate

print(f"Sharpe Ratio with No Short-Selling: {best_sharpe:.4f}")

Sharpe Ratio with No Short-Selling: 0.1124


c)

In [20]:
rf = 0.002
re_M = 0.008  # 0.8% per month
q = 0.05
N = 50
T = 360
num_simulations = 1000

simulated_datasets = []

for sim in range(num_simulations):
    beta = np.random.uniform(0.5, 1.5, N)
    alpha = np.random.uniform(0.1, 0.3, N)
    
    factor_returns = np.random.normal(0, q, T)
    
    returns_sim = np.zeros((T, N))
    for n in range(N):
        epsilon = np.random.normal(0, 1, T)
        returns_sim[:, n] = (rf + re_M * beta[n]) + beta[n] * factor_returns + alpha[n] * epsilon

    mu_pop_sim = rf + re_M * beta
    Sigma_pop_sim = np.diag(alpha**2) + q**2 * np.outer(beta, beta)

    simulated_datasets.append({
        "returns": returns_sim,
        "mu_pop": mu_pop_sim,
        "Sigma_pop": Sigma_pop_sim
    })

In [21]:
ones_N = np.ones(N)

def minimize_variance(target_mu, mu_estimate, Sigma_estimate):
    def portfolio_variance(w):
        return w @ Sigma_estimate @ w
    constraints = [
        {'type': 'eq', 'fun': lambda w: w.sum() - 1},
        {'type': 'eq', 'fun': lambda w: w @ mu_estimate - target_mu}
    ]
    bounds = [(0, None)] * N
    initial_w = ones_N / N
    result = minimize(portfolio_variance, initial_w, method='SLSQP',
                      bounds=bounds, constraints=constraints,
                      options={'maxiter': 200, 'ftol': 1e-6})
    return result.x if result.success else None

def process_simulation(sim_data):
    returns = sim_data["returns"]
    mu_pop, Sigma_pop = sim_data["mu_pop"], sim_data["Sigma_pop"]

    mu_sample = returns.mean(axis=0)
    Sigma_sample = np.cov(returns, rowvar=False)
    inv_Sigma_sample = np.linalg.inv(Sigma_sample)

    market_weights = ones_N / N
    market = returns @ market_weights

    # i.
    sharpe_market = (market.mean() - rf) / market.std(ddof=0)

    # ii.
    sigma_1N = np.sqrt(market_weights @ Sigma_pop @ market_weights)
    sharpe_1N = (market_weights @ mu_pop - rf) / sigma_1N

    # iii.
    num_mv = inv_Sigma_sample @ mu_sample
    w_mv = num_mv / (ones_N @ num_mv)
    sigma_mv = np.sqrt(w_mv @ Sigma_pop @ w_mv)
    sharpe_mv = (w_mv @ mu_pop - rf) / sigma_mv

    # iv.
    num_min = inv_Sigma_sample @ ones_N
    w_min = num_min / (ones_N @ num_min)
    sigma_min = np.sqrt(w_min @ Sigma_pop @ w_min)
    sharpe_min = (w_min @ mu_pop - rf) / sigma_min

    # v.
    mu_0 = (ones_N @ num_mv) / (ones_N @ num_min)
    diff = mu_sample - mu_0
    v = (N + 2) / (N + 2 + T * diff @ inv_Sigma_sample @ diff)
    mu_jsj = (1 - v) * mu_sample + v * mu_0
    num_jsj = inv_Sigma_sample @ mu_jsj
    w_jsj = num_jsj / (ones_N @ num_jsj)
    sigma_jsj = np.sqrt(w_jsj @ Sigma_pop @ w_jsj)
    sharpe_jsj = (w_jsj @ mu_pop - rf) / sigma_jsj

    # vi.
    market_centered = market - market.mean()
    market_var = np.var(market, ddof=0)
    returns_centered = returns - returns.mean(axis=0)
    betas = returns_centered.T @ market_centered / (T * market_var)
    mu_CAPM = rf + betas * (market.mean() - rf)
    num_capm = inv_Sigma_sample @ mu_CAPM
    w_CAPM = num_capm / (ones_N @ num_capm)
    sigma_CAPM = np.sqrt(w_CAPM @ Sigma_pop @ w_CAPM)
    sharpe_CAPM = (w_CAPM @ mu_pop - rf) / sigma_CAPM

    # vii.
    best_sharpe_cmv = -np.inf
    target_returns = np.linspace(np.percentile(mu_sample, 5), np.percentile(mu_sample, 95), 50)
    for target_mu in target_returns:
        w_cmv = minimize_variance(target_mu, mu_sample, Sigma_sample)
        if w_cmv is not None:
            mu_true = w_cmv @ mu_pop
            sigma_true = np.sqrt(w_cmv @ Sigma_pop @ w_cmv)
            sharpe_candidate = (mu_true - rf) / sigma_true
            if sharpe_candidate > best_sharpe_cmv:
                best_sharpe_cmv = sharpe_candidate

    return [
        sharpe_market,
        sharpe_1N,
        sharpe_mv,
        sharpe_min,
        sharpe_jsj,
        sharpe_CAPM,
        best_sharpe_cmv
    ]

In [22]:
start = time.time()

# Run simulations in parallel
results = Parallel(n_jobs=-1, backend='loky')(
    delayed(process_simulation)(sim) for sim in tqdm(simulated_datasets, desc="Simulating", unit="sim")
)

end = time.time()
print(f"\nCompleted in {end - start:.2f} seconds.")

# Unpack results
sharpe_market_list, sharpe_1N_list, sharpe_mv_list = [], [], []
sharpe_min_list, sharpe_jsj_list, sharpe_CAPM_list, sharpe_cmv_list = [], [], [], []

for res in results:
    s_m, s_1N, s_mv, s_min, s_jsj, s_capm, s_cmv = res
    sharpe_market_list.append(s_m)
    sharpe_1N_list.append(s_1N)
    sharpe_mv_list.append(s_mv)
    sharpe_min_list.append(s_min)
    sharpe_jsj_list.append(s_jsj)
    sharpe_CAPM_list.append(s_capm)
    sharpe_cmv_list.append(s_cmv)


Completed in 5324.69 seconds.


Simulating: 100%|██████████| 1000/1000 [1:28:24<00:00,  5.30s/sim]


In [23]:
print("\nAverage Sharpe Ratios (1000 Simulations):")
print(f"Market Portfolio: {np.mean(sharpe_market_list):.4f}")
print(f"1/N Portfolio: {np.mean(sharpe_1N_list):.4f}")
print(f"Mean-Variance Optimal Portfolio: {np.mean(sharpe_mv_list):.4f}")
print(f"Global Minimum Variance Portfolio: {np.mean(sharpe_min_list):.4f}")
print(f"James-Stein-Jorion Estimator: {np.mean(sharpe_jsj_list):.4f}")
print(f"CAPM-based Portfolio: {np.mean(sharpe_CAPM_list):.4f}")
print(f"No Short-Selling Portfolio: {np.mean(sharpe_cmv_list):.4f}")


Average Sharpe Ratios (1000 Simulations):
Market Portfolio: 0.1421
1/N Portfolio: 0.1378
Mean-Variance Optimal Portfolio: 0.0587
Global Minimum Variance Portfolio: 0.1117
James-Stein-Jorion Estimator: 0.0811
CAPM-based Portfolio: 0.1356
No Short-Selling Portfolio: 0.1241
